# Uniform Cost Search

Implementation of the Uniform-Cost search algorithm.

In [1]:
from queue import Queue, PriorityQueue
import numpy as np
from enum import Enum

https://wiki.python.org/moin/TimeComplexity gives a solid overview of Python data structures and their time complexity.

[`Enum`](https://docs.python.org/3/library/enum.html#module-enum) is used to represent possible actions on the grid.


[`PriorityQueue`](https://docs.python.org/3/library/queue.html#queue.PriorityQueue) is used in order to prioritize expanding nodes with lower cost. More about `PriorityQueue` [here](https://www.linode.com/docs/guides/python-priority-queue/).

In [2]:
class Action(Enum):
    """
    An action is represented by a 3 element tuple.
    
    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    """
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1.5)      # Up action is set to cost more than the others
    DOWN = (1, 0, 1)
    
    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
    
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])
            
    
def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN]
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = current_node
    
    # check if the node is off the grid or
    # it's an obstacle
    
    if x - 1 < 0 or grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 > n or grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if y - 1 < 0 or grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if y + 1 > m or grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)
        
    return valid

def visualize_path(grid, path, start):
    sgrid = np.zeros(np.shape(grid), dtype=np.str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    
    for a in path:
        da = a.value
        sgrid[pos[0], pos[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'  
    return sgrid

### Cost Search

In this section the breadth-first search algorithm is extended by incorporating a cost for each action. In order to implement the Uniform Cost Search the lowest cost path is be selected at every expansion thanks to the PriorityQueue class, that priorizes the value of the first argument in every element of the queue, to take the lowest.

In [3]:
def uniform_cost(grid, start, goal):

    # Initialize queue, visited list and branch dictionay
    path = []
    q = PriorityQueue()     # Priority queue for prioritizing expanding cells with lower cost
    q.put((0, start))       # (cost, cell); cost value is use for priority
    visited = set(start)
    branch = {}
    
    found = False           # Fag for end of search
    
    # Run loop while queue is not empty
    while not q.empty():
        # Get and remove the first element from the queue
        current_cost, current_node = q.get()

        # If the current cell corresponds to the goal state, stop the search
        if current_node == goal:        
            print('Found a path.')
            found = True
            break
        else:
            # Get the new valid nodes connected to the current node
            valid = valid_actions(grid, current_node) 
            # Iterate through each of the new nodes and:
            for action in valid:
                da = action.delta    # delta-movement of performing the action
                cost = action.cost   # cost of performing the action
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                new_cost = current_cost + cost
                
                # If the node has not been visited you will need to
                if next_node not in visited:      # If the node has not been visited
                    visited.add(next_node)        # mark it as visited
                    q.put((new_cost, next_node))  # add it to the queue          
                    branch[next_node] = (new_cost, current_node, action) # add how you got there and the cost of getting there
             
    path = []
    path_cost = 0
    if found:
        
        # retrace steps
        path = []
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
            
    return path[::-1], path_cost

### Executing the search

Run `uniform_cost()` and reference the grid to see if the path makes sense.

Note that the 2D-grid world is symmetric and the start and goal cells keep that symmetry so that there are two possible paths for getting the goal but only one gives minimum cost when the UP-action has a higher cost than the LOW-action.

In [7]:
start = (2, 0)
goal = (2, 5)

grid = np.array([
    [0, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0],
])

In [8]:
path, path_cost = uniform_cost(grid, start, goal)
print('Path cost: ', path_cost)
print(path)

Found a path.
Path cost:  10.0
[<Action.DOWN: (1, 0, 1)>, <Action.DOWN: (1, 0, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.UP: (-1, 0, 1.5)>, <Action.UP: (-1, 0, 1.5)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>]


In [9]:
# S -> start, G -> goal, O -> obstacle
visualize_path(grid, path, start)

array([[' ', ' ', ' ', 'O', ' ', ' '],
       [' ', 'O', ' ', 'O', ' ', ' '],
       ['S', 'O', '>', '>', '>', 'G'],
       ['v', 'O', '^', 'O', ' ', ' '],
       ['>', '>', '^', 'O', ' ', ' ']], dtype='<U1')